## Imports Librairies

In [ ]:
import ultralytics, cv2, os, yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
ultralytics.checks()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Création fichier YAML

Ce fichier donne quelques paramètres comme le chemin d'accès aux données d’entraînement et de validation ainsi que le(s) label(s) à détecter

In [ ]:
cell_yaml = dict(
    train = r'C:datasets\images\train',
    val = r'datasets\images\val',
    test = r'datasets\images\test',
    nc = 1,
    names = ['cell']
)

with open('datasets/cell.yaml', 'w') as outfile:
    yaml.dump(cell_yaml, outfile, default_flow_style=True)

## Entraînement

### Chargement du Modèle Yolo

In [ ]:
model_name = 'yolov8l.pt'

model = YOLO(model_name)

### Entraînement sur notre dataset

In [ ]:
model.train(data = 'datasets/cell.yaml', 
            epochs = 50,
            patience = 20, 
            imgsz = 640, 
            dropout = 0.2,
            project = model_name[:-3], 
            device = 'cpu',
            verbose = True)

## Test

In [ ]:
model_name = 'yolov8l.pt'

In [ ]:
# Chemins du modèle entraîné et des images de test
best_path = model_name[:-3] + '/train/weights/best.pt'
source = 'datasets/images/test_x40'

In [ ]:
# Test des images à tester
ppaths=[]
for dirname, _, filenames in os.walk(source):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths += [(os.path.join(dirname, filename))]
            
ppaths = sorted(ppaths)
print(ppaths)

In [ ]:
# Chargement du modèle
model_trained = YOLO(best_path)

In [ ]:
# Intervalle de confiance (de 0 à 1)
confiance = 0.05

# Prédiction
results = model_trained(source, conf = confiance)

print(len(results[0]))
# Sauvegarde des images annotées
for i, result in enumerate(results):
    plot = result.plot(line_width = 1, font_size = 2)
    cv2.imwrite(f'result_{model_name[:-3]}_{i}.jpg', plot)
